# Load data

In [1]:
import os
import pandas as pd
from collections import defaultdict , Counter
import torch
import torch.nn as nn
from torch.nn.functional import cross_entropy
from datasets import Dataset , DatasetDict , Sequence , Value , Features , ClassLabel
from transformers import AutoTokenizer , XLMRobertaConfig , AutoConfig , TrainingArguments , DataCollatorForTokenClassification , Trainer
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel
from seqeval.metrics import f1_score
from sklearn.metrics import ConfusionMatrixDisplay , confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_from_disk
import warnings
from transformers import AutoTokenizer
from pathlib import Path


warnings.filterwarnings('ignore')
data = load_from_disk("ner_dataset_fa_en.hf")

In [2]:
data = data.shuffle(seed=42)

Loading cached shuffled indices for dataset at E:\ML\NLP_Toolbox\NER\ner_dataset_fa_en.hf\train\cache-d1f77b80321dd72f.arrow
Loading cached shuffled indices for dataset at E:\ML\NLP_Toolbox\NER\ner_dataset_fa_en.hf\test\cache-0a5324390e9e73f0.arrow
Loading cached shuffled indices for dataset at E:\ML\NLP_Toolbox\NER\ner_dataset_fa_en.hf\validation\cache-582f6df3cfde4ff7.arrow


In [3]:
roberta_model_name = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)

# Create custom model

In [4]:
class ParsXLMRobertaForTokenClassification(RobertaPreTrainedModel):
    
    class_config = XLMRobertaConfig
    
    def __init__(self , config):
        
        super().__init__(config)
        
        self.num_labels = config.num_labels
        
        self.roberta = RobertaModel(config , add_pooling_layer=False)
        
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
        self.classifier = nn.Linear(config.hidden_size , config.num_labels)
        
        self.init_weights()
        
    def forward(self , input_ids=None , attention_mask=None , token_type_ids=None , labels=None , **kwargs):

        outputs = self.roberta(input_ids , attention_mask=attention_mask , token_type_ids=token_type_ids , **kwargs)

        sequence_output = self.dropout(outputs[0])

        logits = self.classifier(sequence_output)

        loss = None

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1 , self.num_labels) , labels.view(-1))

        return TokenClassifierOutput(
            loss = loss,
            logits = logits,
            hidden_states = outputs.hidden_states,
            attentions = outputs.attentions
        )

# EDA and data collections

In [5]:
ner_tags = data['train']['ner_tags_names']
ner_tag_names = set(tag for tags in ner_tags for tag in tags)
                
index2tag = {idx: tag for idx, tag in enumerate(ner_tag_names)}
tag2index = {tag: idx for idx, tag in enumerate(ner_tag_names)}

In [6]:
import pandas as pd

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
per_text = " ".join(data['train']['tokens'][0])
input_ids = tokenizer.encode(per_text, return_tensors="pt")
roberta_tokens = tokenizer(per_text).tokens()
pd.DataFrame([roberta_tokens, input_ids[0].numpy()], index=["Tokens", "Input IDs"])

,0,1,2,3,4,5,6,7,8
Tokens,<s>,▁NEW,▁,YO,RK,▁1996,-08-,24,</s>
Input IDs,0,80111,6,32256,31875,11891,47355,2357,2


# Tokenzier and aligning 

In [7]:
from transformers import RobertaTokenizerFast
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name, add_prefix_space=True)

def tokenize_and_align_labels(examples):
    tokenized_inputs = roberta_tokenizer(examples["tokens"], truncation=True,
    is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples["ner_tags_names"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                #  if a word convert more than one toekn, ignore except first token
                label_ids.append(-100)
            else:
                label_token = label[word_idx]
                # Use the label map to get the numerical value for each entity
                label_ids.append(tag2index[label_token])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [8]:
# apply fucntion
def encode(corpus):
    return corpus.map(tokenize_and_align_labels,batched=True,
                      remove_columns=['tokens' ,'__index_level_0__', 'ner_tags' , 'ner_tags_names'])

In [9]:
encoded_data = encode(data)

Map:   0%|          | 0/33181 [00:00<?, ? examples/s]

Map:   0%|          | 0/5583 [00:00<?, ? examples/s]

Map:   0%|          | 0/5334 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoConfig

roberta_config  = AutoConfig.from_pretrained(
    roberta_model_name,
    num_labels = len(index2tag),
    id2label = index2tag,
    label2id = tag2index
)

In [11]:
encoded_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 33181
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5583
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5334
    })
})

In [12]:
import numpy as np

def align_predictions(predictions, label_ids):
        preds = np.argmax(predictions, axis=2)
        batch_size, seq_len = preds.shape
        labels_list, preds_list = [], []
        for batch_idx in range(batch_size):
            example_labels, example_preds = [], []
            for seq_idx in range(seq_len):
                # Ignore label IDs = -100
                if label_ids[batch_idx, seq_idx] != -100:
                    example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                    example_preds.append(index2tag[preds[batch_idx][seq_idx]])
            labels_list.append(example_labels)
            preds_list.append(example_preds)
        return preds_list, labels_list

# Training

In [28]:
from transformers import TrainingArguments
import torch

# init training args
num_epochs = 4
batch_size = 24
logging_steps = len(encoded_data['train']) // batch_size

training_args = TrainingArguments(
    output_dir="Roberta-fa-en-ner", log_level="error", num_train_epochs=num_epochs,
    gradient_checkpointing=True,
    eval_accumulation_steps=10,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    seed=42,
    logging_strategy="steps", evaluation_strategy="epoch",
    save_steps=1e6, weight_decay=0.01, disable_tqdm=False,
    logging_steps=logging_steps, push_to_hub=False)


In [29]:
# function for model
def model_init():
    return (ParsXLMRobertaForTokenClassification
                  .from_pretrained(roberta_model_name, config=roberta_config,cache_dir=Path.cwd())
                  .to(device))

In [30]:
from transformers import DataCollatorForTokenClassification
# Use the custom data collator
data_collator = DataCollatorForTokenClassification(tokenizer=roberta_tokenizer)

In [31]:
from seqeval.metrics import f1_score,recall_score,precision_score,accuracy_score
import wandb

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions,
    eval_pred.label_ids)
    wandb.log({"f1": f1_score(y_true, y_pred),"Recall":recall_score(y_true, y_pred),"Precision":precision_score(y_true, y_pred),"Accuracy":accuracy_score(y_true, y_pred)})
    return {"f1": f1_score(y_true, y_pred),"Recall":recall_score(y_true, y_pred),"Precision":precision_score(y_true, y_pred),"Accuracy":accuracy_score(y_true, y_pred)}

In [1]:
from transformers import Trainer
import torch

# training phase
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trainer = Trainer(model_init=model_init, args=training_args,
                    data_collator=data_collator, compute_metrics=compute_metrics,
                    train_dataset=encoded_data['train'],
                    eval_dataset=encoded_data['validation'],
                    tokenizer=tokenizer)


In [ ]:
import wandb

# monitoring
wandb.init(project="XLMRoberta_not_aligned_data",name='training-on-english-persian')
result = trainer.train()
wandb.finish()

# Testing

In [ ]:
# function for testing
def tag_text(text,tags,tokenizer):
    
    tokens = tokenizer(text).tokens()    
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
    outputs = trainer.model(input_ids)[0]
    predictions = torch.argmax(outputs, dim=2)    
    preds = [tags[p] for p in predictions[0].cpu().numpy()]
    return pd.DataFrame([tokens , preds] , index=['token' , 'predicted label'])

In [ ]:
text = "سورنا در ایران زندگی میکرد."
tag_text(text , list(ner_tag_names) , tokenizer)

In [ ]:
trainer

In [ ]:
" ".join(data['train'][120]['tokens'])

In [ ]:
data['train'][120]['tokens'][0]